this code for compare diffrent types of feature selection techniques.
experiment 3: Comparison between the probosed hybrid model (univariate ROC-AUC + ReliefF + SVM-RFE, and three different combi- nations of feature selection techniques (Mutual+ SVM-RFE. ReliefF+ SVM-RFE, and Uni+ SVM-RFE).

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV, cross_val_score, train_test_split, LeaveOneOut, KFold, StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, classification_report
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression, Lasso
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier, GradientBoostingClassifier, ExtraTreesClassifier
import xgboost as xgb
from feature_engine.selection import DropDuplicateFeatures, DropConstantFeatures
from sklearn.feature_selection import SelectFromModel, VarianceThreshold
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN, BorderlineSMOTE, SVMSMOTE
from sklearn.ensemble import VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from mlxtend.classifier import EnsembleVoteClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score, plot_roc_curve
from yellowbrick.classifier import ROCAUC


In [ ]:
from sklearn.feature_selection import RFE

# input the data (gene expression array) and labels

In [ ]:
filenames = ['riaz', 'gide', 'van allen', 'hugo', 'lee']
name= filenames[1]
name

In [ ]:
filenames = ['riaz', 'gide', 'van allen', 'hugo', 'lee']
name= filenames[1]

# Read logged data
logged_data = pd.read_csv('data/' + name + '_logged_data.csv', index_col='Unnamed: 0').T
print(logged_data.shape)
print(logged_data.head())

# Read labels
label = pd.read_csv('data/' + name + '.Pre.Samples.corr.csv', index_col='Unnamed: 0')
print(label.shape)
print(label.head())

# Count of labels
print(label['Resp_NoResp'].value_counts())

# Normalized count of labels
print(label['Resp_NoResp'].value_counts(normalize=True))

# Label encoding
y = label.replace({'No_Response': 0, 'Response': 1})
labels = ['No_Response', 'Response']  # for plotting convenience later on

print(y.head())
print(y['Resp_NoResp'].head())
print(y.iloc[:, 0].head())
print(y.iloc[:, 0].shape)


In [ ]:
#read the immune related genes from immport
IRG= pd.read_csv('IRG.csv')
print(IRG.shape)
IRG.head()

# data oversampling

In [ ]:
# Define features (X) and target (Y)
X = logged_data
Y = y.iloc[:, 0]

# Oversampling using BorderlineSMOTE
ada = BorderlineSMOTE(
    sampling_strategy='auto',  # samples only the minority class
    random_state=0,  # for reproducibility
    k_neighbors=5,
    m_neighbors=10,
    kind='borderline-1',
    n_jobs=4
)

# Resample the data
X_res, y_res = ada.fit_resample(X, Y)

# Print the shapes of resampled data
print(X_res.shape, y_res.shape)

# Print the counts of the original and resampled target variable
print(Y.value_counts(), y_res.value_counts())


# separate dataset into train and test


In [ ]:
# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.3, random_state=0)

# Display shapes of train and test sets
print(X_train.shape, X_test.shape)

# Display first few rows of test set
print(X_test.head())

# Display normalized value counts of target variable in test set
print(y_test.value_counts(normalize=True))

# Keep a copy of the original datasets
X_train_original = X_train.copy()
X_test_original = X_test.copy()

# Check shapes of train and test sets after copying
print(X_train.shape, X_test.shape)


# result of the probosed hybrid model (univariate ROC-AUC + ReliefF + SVM-RFE), use of the data selected by Univariate ROC-AUC Performance + ReliefF from experiment 2

In [ ]:
X_train = pd.read_csv('gide_X_train_uni_reliefF.csv',index_col = 'Unnamed: 0')
print(X_train.shape)
X_train.head()


In [ ]:
X_test = pd.read_csv('gide_X_test_uni_reliefF.csv',index_col = 'Unnamed: 0')
print(X_test.shape)
X_test.head()

# SVM-RFE

In [ ]:
from sklearn.ensemble import VotingClassifier
accuracy_rate = []
clf1 = ExtraTreesClassifier()
clf2 = LogisticRegression(penalty ='l2', C=0.1, solver='liblinear', random_state=4,max_iter = 1000)
clf3 = SVC(gamma =1e-05, probability = True, decision_function_shape = 'ovo', kernel = 'linear')
clf4 = AdaBoostClassifier()
clf5= XGBClassifier()



# Will take some time
for i in range(1, 51, 1):
    
    svm= SVC(gamma =1e-05, probability = True, decision_function_shape = 'ovo', kernel = 'linear')
    sel_ = RFE(svm, n_features_to_select=i)
    sel_.fit(X_train, y_train)
    selected_feat = X_train.columns[(sel_.get_support())]
    print(len(selected_feat))
    X_train_rfe = X_train[selected_feat]
    X_test_rfe = X_test[selected_feat]
    # display features
    
    
    
    eclf1 = VotingClassifier(estimators=[ ('logistic', clf2),('SVM', clf3),
                                           ('XGB', clf5), 
                                          ('extratree',clf1), ('AdaBoost', clf4)
                                        ], voting='hard')    
    print(i)
    
    for clf in (clf1,clf2, clf3, clf4, clf5, eclf1):
        clf.fit(X_train_rfe, y_train)
        y_pred = clf.predict(X_test_rfe)
        print(clf.__class__.__name__, round(accuracy_score(y_test, y_pred),3))
    
    eclf1.fit(X_train_rfe, y_train)
    predictions = eclf1.predict(X_test_rfe)
    print(classification_report(y_test, predictions))
    print(X_train_rfe.columns)
    accuracy_rate.append(round(accuracy_score(y_test, predictions),3))



In [ ]:
accuracy_rate

In [ ]:
plt.figure(figsize=(20,6))
plt.plot(range(1,51,1),accuracy_rate,color='blue', linestyle='dashed', marker='o',
         markerfacecolor='red', markersize=10)
plt.title('uni+ relieff + SVM-RFE')
plt.xlabel('No. of Genes')
plt.ylabel('ReliefF Accuracy')

In [ ]:
# save result in a data frame for ploting
accuracy_plot_new = pd.DataFrame()
accuracy_plot_new['uni+ relieff + RFE'] = accuracy_rate

# result of the (univariate ROC-AUC + SVM-RFE) model, use of the data selected by Univariate ROC-AUC Performance from experiment 1

In [ ]:
X_train = pd.read_csv('gide_X_train_uni_roc-auc.csv',index_col = 'Unnamed: 0')
print(X_train.shape)
X_train.head()


In [ ]:
X_test = pd.read_csv('gide_X_test_uni_roc-auc.csv',index_col = 'Unnamed: 0')
print(X_test.shape)
X_test.head()

# SVM-RFE

In [ ]:
from sklearn.ensemble import VotingClassifier
accuracy_rate = []
clf1 = ExtraTreesClassifier()
clf2 = LogisticRegression(penalty ='l2', C=0.1, solver='liblinear', random_state=4,max_iter = 1000)
clf3 = SVC(gamma =1e-05, probability = True, decision_function_shape = 'ovo', kernel = 'linear')
clf4 = AdaBoostClassifier()
clf5= XGBClassifier()



# Will take some time
for i in range(1, 51, 1):
    
    svm= SVC(gamma =1e-05, probability = True, decision_function_shape = 'ovo', kernel = 'linear')
    sel_ = RFE(svm, n_features_to_select=i)
    sel_.fit(X_train, y_train)
    selected_feat = X_train.columns[(sel_.get_support())]
    print(len(selected_feat))
    X_train_uni_rfe = X_train[selected_feat]
    X_test_uni_rfe = X_test[selected_feat]
    # display features
    
    
    
    eclf1 = VotingClassifier(estimators=[ ('logistic', clf2),('SVM', clf3),
                                           ('XGB', clf5), 
                                          ('extratree',clf1), ('AdaBoost', clf4)
                                        ], voting='hard')    
    print(i)
    
    for clf in (clf1,clf2, clf3, clf4, clf5, eclf1):
        clf.fit(X_train_uni_rfe, y_train)
        y_pred = clf.predict(X_test_uni_rfe)
        print(clf.__class__.__name__, round(accuracy_score(y_test, y_pred),3))
    
    eclf1.fit(X_train_uni_rfe, y_train)
    predictions = eclf1.predict(X_test_uni_rfe)
    print(classification_report(y_test, predictions))
    print(X_train_uni_rfe.columns)
    accuracy_rate.append(round(accuracy_score(y_test, predictions),3))



In [ ]:
accuracy_rate

In [ ]:
plt.figure(figsize=(20,6))
plt.plot(range(1,51,1),accuracy_rate,color='blue', linestyle='dashed', marker='o',
         markerfacecolor='red', markersize=10)
plt.title('uni + SVM-RFE')
plt.xlabel('No. of Genes')
plt.ylabel('ReliefF Accuracy')

In [ ]:
# save result in a data frame 'accuracy_plot_new'

accuracy_plot_new['uni + RFE'] = accuracy_rate

# result of the (Relieff + SVM-RFE) model, use of the data selected by Relieff from experiment 1

In [ ]:
X_train = pd.read_csv('gide_X_train_reliefF.csv',index_col = 'Unnamed: 0')
print(X_train.shape)
X_train.head()


In [ ]:
X_test = pd.read_csv('gide_X_test_reliefF.csv',index_col = 'Unnamed: 0')
print(X_test.shape)
X_test.head()

# SVM-RFE

In [ ]:
from sklearn.ensemble import VotingClassifier
accuracy_rate = []
clf1 = ExtraTreesClassifier()
clf2 = LogisticRegression(penalty ='l2', C=0.1, solver='liblinear', random_state=4,max_iter = 1000)
clf3 = SVC(gamma =1e-05, probability = True, decision_function_shape = 'ovo', kernel = 'linear')
clf4 = AdaBoostClassifier()
clf5= XGBClassifier()



# Will take some time
for i in range(1, 51, 1):
    
    svm= SVC(gamma =1e-05, probability = True, decision_function_shape = 'ovo', kernel = 'linear')
    sel_ = RFE(svm, n_features_to_select=i)
    sel_.fit(X_train, y_train)
    selected_feat = X_train.columns[(sel_.get_support())]
    print(len(selected_feat))
    X_train_rf_rfe = X_train[selected_feat]
    X_test_rf_rfe = X_test[selected_feat]
    # display features
    
    
    
    eclf1 = VotingClassifier(estimators=[ ('logistic', clf2),('SVM', clf3),
                                           ('XGB', clf5), 
                                          ('extratree',clf1), ('AdaBoost', clf4)
                                        ], voting='hard')    
    print(i)
    
    for clf in (clf1,clf2, clf3, clf4, clf5, eclf1):
        clf.fit(X_train_rf_rfe, y_train)
        y_pred = clf.predict(X_test_rf_rfe)
        print(clf.__class__.__name__, round(accuracy_score(y_test, y_pred),3))
    
    eclf1.fit(X_train_rf_rfe, y_train)
    predictions = eclf1.predict(X_test_rf_rfe)
    print(classification_report(y_test, predictions))
    print(X_train_rf_rfe.columns)
    accuracy_rate.append(round(accuracy_score(y_test, predictions),3))



In [ ]:
accuracy_rate

In [ ]:
plt.figure(figsize=(20,6))
plt.plot(range(1,51,1),accuracy_rate,color='blue', linestyle='dashed', marker='o',
         markerfacecolor='red', markersize=10)
plt.title(' relieff + SVM-RFE')
plt.xlabel('No. of Genes')
plt.ylabel('ReliefF Accuracy')

In [ ]:
# save result in a data frame 'accuracy_plot_new'

accuracy_plot_new['relieff + RFE'] = accuracy_rate

# result of the (Mutual information + SVM-RFE) model, use of the data selected by Relieff from experiment 1

In [ ]:
X_train = pd.read_csv('gide_X_train_MI.csv',index_col = 'Unnamed: 0')
print(X_train.shape)
X_train.head()


In [ ]:
X_test = pd.read_csv('gide_X_test_MI.csv',index_col = 'Unnamed: 0')
print(X_test.shape)
X_test.head()

# SVM-RFE

In [ ]:
from sklearn.ensemble import VotingClassifier
accuracy_rate = []
clf1 = ExtraTreesClassifier()
clf2 = LogisticRegression(penalty ='l2', C=0.1, solver='liblinear', random_state=4,max_iter = 1000)
clf3 = SVC(gamma =1e-05, probability = True, decision_function_shape = 'ovo', kernel = 'linear')
clf4 = AdaBoostClassifier()
clf5= XGBClassifier()



# Will take some time
for i in range(1, 51, 1):
    
    svm= SVC(gamma =1e-05, probability = True, decision_function_shape = 'ovo', kernel = 'linear')
    sel_ = RFE(svm, n_features_to_select=i)
    sel_.fit(X_train, y_train)
    selected_feat = X_train.columns[(sel_.get_support())]
    print(len(selected_feat))
    X_train_mi_rfe = X_train[selected_feat]
    X_test_mi_rfe = X_test[selected_feat]
    # display features
    
    
    
    eclf1 = VotingClassifier(estimators=[ ('logistic', clf2),('SVM', clf3),
                                           ('XGB', clf5), 
                                          ('extratree',clf1), ('AdaBoost', clf4)
                                        ], voting='hard')    
    print(i)
    
    for clf in (clf1,clf2, clf3, clf4, clf5, eclf1):
        clf.fit(X_train_mi_rfe, y_train)
        y_pred = clf.predict(X_test_mi_rfe)
        print(clf.__class__.__name__, round(accuracy_score(y_test, y_pred),3))
    
    eclf1.fit(X_train_mi_rfe, y_train)
    predictions = eclf1.predict(X_test_mi_rfe)
    print(classification_report(y_test, predictions))
    print(X_train_mi_rfe.columns)
    accuracy_rate.append(round(accuracy_score(y_test, predictions),3))



In [ ]:
accuracy_rate

In [ ]:
plt.figure(figsize=(20,6))
plt.plot(range(1,51,1),accuracy_rate,color='blue', linestyle='dashed', marker='o',
         markerfacecolor='red', markersize=10)
plt.title(' relieff + SVM-RFE')
plt.xlabel('No. of Genes')
plt.ylabel('ReliefF Accuracy')

In [ ]:
# save result in a data frame 'accuracy_plot_new'

accuracy_plot_new['mutual + RFE'] = accuracy_rate

In [ ]:
plt.figure(figsize=(20,6))
plt.plot(range(1,51,1),accuracy_plot_new['mutual + RFE'],color='blue', linestyle='solid', marker='o',
         markerfacecolor='blue', markersize=5, label = "mutual + SVM-RFE")
plt.plot(range(1,51,1),accuracy_plot_new['uni + RFE'],color='green', linestyle='solid',  marker='o',
         markerfacecolor='green', markersize=5, label = "uni + SVM-RFE")
plt.plot(range(1,51,1),accuracy_plot_new['relieff + RFE'],color='red', linestyle='solid', marker='o',
         markerfacecolor='red', markersize=5, label = "reliefF + SVM-RFE")
plt.plot(range(1,51,1),accuracy_plot_new['uni+ relieff + RFE'],color='black', linestyle='solid', marker='o',
         markerfacecolor='black', markersize=7, label = "uni+ reliefF + SVM-RFE")

#plt.ylim(0.4, 1)
#plt.title('ReliefF and mutual info Accuracies vs. No. of Genes')
plt.xlabel('No. of Genes', fontsize=16)
plt.ylabel(' Accuracy', fontsize=16)
plt.legend()
plt.gca().invert_xaxis()
plt.savefig('four curves_gide.jpg')
plt.savefig('four curves_gide.png')
plt.savefig('four curves_gide.eps',dpi=300)
plt.savefig('four curves_gide.pdf',dpi=300)